In [1]:
import json, requests

# custom information
CLIENT_ID = " "

In [2]:
def download_mapillary_image_information(url: str, file_path: str=None) -> dict:
    """download information for mapillary data for a given url
    and save these metadata to a given file path (json format)
    - inspired from: 
    https://blog.mapillary.com/update/2020/02/06/mapillary-data-in-jupyter.html

    Args:
        url (str): url to request information for 
        file_path (str, Optional): path to json file to save informaiton in 

    Returns:
        dict: dictionary containing all output data in geo json format
    """
    # create an empty GeoJSON to collect all images we find
    output = {"type":"FeatureCollection","features":[]}
    print("Request URL: {}".format(url))

    # get the request with no timeout in case API is slow
    r = requests.get(url, timeout=None)

    # check if request failed, if failed, keep trying 
    while r.status_code != 200:
        r = requests.get(url, timeout=200)
        print("Request failed")

    # retrieve data and save them to output dict
    data = r.json() 
    data_length = len(data['features']) 
    for feature in data['features']:
        output['features'].append(feature) 

    if data_length == 0:
        print("No data available for the request")

    # if we receive 500 items, there should be a next page
    while data_length == 500:

        # get the URL for a next page
        link = r.links['next']['url']

        # retrieve the next page in JSON format
        r = requests.get(link)

        # try again if the request fails
        while r.status_code != 200:
            r = requests.get(url, timeout=200)

        # retrieve data and save them to output dict
        data = r.json()
        for feature in data['features']:
            output['features'].append(feature)

        data_length = len(data['features']) #update data length
        print('Total images: {}'.format(len(output['features']))) 

    # send collected features to the local file
    if file_path is not None:
        with open(file_path, 'w') as outfile: 
            json.dump(output, outfile)

    print('DONE') # once all images are saved in a GeoJSON and saved, we finish
    print('Total images: {}'.format(len(output['features']))) 

    return output


In [3]:
## download data close to a certain point

# location to search images close to; longitude,latitude
close_to = "9.1713443962406,48.78071364962406" 

# metres around close_to to search images for
radius = 100 

# sort_by=key enables pagination
url = ('https://a.mapillary.com/v3/images?client_id={}&closeto={}&radius={}&per_page=500&sort_by=key'\
    ).format(CLIENT_ID, close_to, radius) 

# download data from given URL
output_data_closeto = download_mapillary_image_information(url)

Request URL: https://a.mapillary.com/v3/images?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh&closeto=9.1713443962406,48.78071364962406&radius=100&per_page=500&sort_by=key
DONE
Total images: 18


In [4]:
##  download data in a certain bounding box and certain quality score
# Filter by a bounding box on the map, given as min_longitude,min_latitude,max_longitude,max_latitude (lower left, upper right)
bounding_box = "9.175126043956457,48.774203032456604,9.176889,48.77471" 

# define image qualities to consider (1-5 inclusively)
min_score = 3
max_score = 5

# sort_by=key enables pagination
url = ('https://a.mapillary.com/v3/images?client_id={}&bbox={}&per_page=500&sort_by=key&min_quality_score={}&max_quality_score={}'\
    ).format(CLIENT_ID, bounding_box, min_score, max_score) 

# download data from given URL
output_data_bbox = download_mapillary_image_information(url)

Request URL: https://a.mapillary.com/v3/images?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh&bbox=9.175126043956457,48.774203032456604,9.176889,48.77471&per_page=500&sort_by=key&min_quality_score=3&max_quality_score=5
DONE
Total images: 7


In [27]:
## retrieve object detections for certain images
image_segmentations = {}

# iterate over all previously retrieved images
for image in output_data_bbox["features"]:
    image_key = image["properties"]["key"]
    layer = "segmentations"

    # request for object detection layer of a certain image (given by image key)
    url=("https://a.mapillary.com/v3/images/{}/object_detections/{}?client_id={}"\
        ).format(image_key, layer, CLIENT_ID) 
    print("Request URL: {}".format(url))
    r = requests.get(url, timeout=300)
    image_segmentations[image_key] = r.json()

r.json()

Request URL: https://a.mapillary.com/v3/images/pvpEPpNUeB2vqnWHnZX6vV/object_detections/segmentations?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh
Request URL: https://a.mapillary.com/v3/images/jhCVYj6FRzqgtDNCjYsYfq/object_detections/segmentations?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh
Request URL: https://a.mapillary.com/v3/images/VTke5Vw3TrI4ZZiJjNvc1z/object_detections/segmentations?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh
Request URL: https://a.mapillary.com/v3/images/tipybR9iD6b4QpmBqtarZX/object_detections/segmentations?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh
Request URL: https://a.mapillary.com/v3/images/dmKLGS3zHnNr6FjdVWcjFq/object_detections/segmentations?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh
Request URL: https://a.mapillary.com/v3/images/LDvtTcj-5Emzgtj2uMZXIg/object_detections/segmentations?client_id=VldMM2FxNlFTeXRXYkVRY1RQa1lMbjpiNzJmNDdmMmQ2OGYyMzNh
Request UR


   'geometry': {'type': 'Point', 'coordinates': [9.176889, 48.77471]}},
  {'type': 'Feature',
   'properties': {'area': 1.4940897623697916e-05,
    'captured_at': '2020-05-18T18:04:36.000Z',
    'image_ca': 236.00427547525044,
    'image_key': 'OXbgeC5G7CuTOWCLdMFLUw',
    'image_pano': False,
    'key': 'glo47ojedvsrwd3qi36fqb',
    'score': 0.2549019607843137,
    'shape': {'type': 'Polygon',
     'coordinates': [[[0.41455078125, 0.470703125],
       [0.4140625, 0.471435546875],
       [0.41357421875, 0.469970703125],
       [0.41357421875, 0.458984375],
       [0.41455078125, 0.458251953125],
       [0.4150390625, 0.458984375],
       [0.41455078125, 0.470703125]]]},
    'value': 'object--support--pole'},
   'geometry': {'type': 'Point', 'coordinates': [9.176889, 48.77471]}},
  {'type': 'Feature',
   'properties': {'area': 6.0558319091796875e-05,
    'captured_at': '2020-05-18T18:04:36.000Z',
    'image_ca': 236.00427547525044,
    'image_key': 'OXbgeC5G7CuTOWCLdMFLUw',
    'image_

In [8]:
## save Mapillary image files
from urllib.request import urlretrieve

# iterate over all previously retrieved images and save them 
for image in output_data_bbox["features"]:
    image_key = image["properties"]["key"]

    # 2048 = highest resolution; others possible
    url = "https://images.mapillary.com/{}/thumb-2048.jpg".format(image_key) 
    image_local_path =  "{}.jpg".format(image_key)
    urlretrieve(url, image_local_path)